<a href="https://colab.research.google.com/github/Droslj/scATAC-seq-complete-/blob/Google-colab/scATAC_seq_(1)_DA_PyDESeq2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

scATAC seq, based on scATAC seq processing Galaxy tutorials (scATAC preprocessing (2), Standard scATAC seq processing pipeline (1) )
AD Objects created in Galaxy using customized Galaxy WF with Snapatac2 and imported
(1) https://usegalaxy.eu/training-material/topics/single-cell/tutorials/scatac-preprocessing-tenx/tutorial.html#mapping-reads-to-a-reference-genome, (2) https://usegalaxy.eu/training-material/topics/single-cell/tutorials/scatac-standard-processing-snapatac2/tutorial.html
Data taken from the following NCBI study:
Metabolic adaptation pilots the differentiation of human hematopoietic cells (https://www.ncbi.nlm.nih.gov/bioproject/PRJNA1015713)
Import preprocessed Anndata object for four biological replicates, SRR26046013 (cells treated with AOA inhibitor), SRR26046015 (cells treated with DON inhibitor), SRR26046017 (cells treated with DG inhibitor), and SRR26046019 (untreated cells).
Following steps were performed in the preprocessing:
(1) Import matrices
(2) Compute fragment size distribution
(3) Compute TSS enrichment
(4) Filter cell counts based on TSSe
(5) Create cell by bin matrix based on 500 bp wide bins accross the whole genome
(6) Perform feature selection
(7) Perform Doublet removal
(8) Perform Dim reduction (spectral)
(9) Perform Clustering (neighborhood, UMAP, leiden)
(10) Create a cell by gene matrix
(11) Concatenate matrices using Inner join
(12) Remove batch effects

In [1]:
!pip install snapatac2 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 687.2/687.2 kB 9.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all t

In [2]:
!pip show snapatac2

Name: snapatac2
Version: 2.8.0
Summary: SnapATAC2: Single-cell epigenomics analysis pipeline
Home-page: https://github.com/
Author: Kai Zhang <kai@kzhang.org>
Author-email: Kai Zhang <zhangkai33@westlake.edu.cn>
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: anndata, igraph, kaleido, macs3, multiprocess, natsort, numpy, pandas, plotly, polars, pooch, pyarrow, pyfaidx, rustworkx, scikit-learn, scipy, tqdm, typeguard
Required-by: 


In [3]:
import snapatac2 as snap

In [4]:
!pip install umap-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.1 MB/s eta 0:00:00


In [5]:
import umap.umap_ as umap


In [6]:
from umap import UMAP

In [7]:
!pip install scanpy -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 4.7 MB/s eta 0:00:00


In [8]:
import scanpy as sc

In [9]:
pip show scanpy

Name: scanpy
Version: 1.10.4
Summary: Single-Cell Analysis in Python.
Home-page: https://scanpy.org
Author: Alex Wolf, Philipp Angerer, Fidel Ramirez, Isaac Virshup, Sergei Rybakov, Gokcen Eraslan, Tom White, Malte Luecken, Davide Cittaro, Tobias Callies, Marius Lange, Andrés R. Muñoz-Rojas
Author-email: 
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: anndata, h5py, joblib, legacy-api-wrap, matplotlib, natsort, networkx, numba, numpy, packaging, pandas, patsy, pynndescent, scikit-learn, scipy, seaborn, session-info, statsmodels, tqdm, umap-learn
Required-by: 


In [10]:
!pip install pydeseq2 -q # Install PyDESeq2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 1.6 MB/s eta 0:00:00


In [11]:
import pydeseq2

In [12]:
!pip show pydeseq2

Name: pydeseq2
Version: 0.4.12
Summary: A python implementation of DESeq2.
Home-page: 
Author: Boris Muzellec, Maria Telenczuk, Vincent Cabelli and Mathieu Andreux
Author-email: boris.muzellec@owkin.com
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: anndata, matplotlib, numpy, pandas, scikit-learn, scipy
Required-by: 


In [13]:
import numpy as np

In [14]:
import anndata as ad

In [15]:
import matplotlib.pyplot as plt

In [16]:
import seaborn as sns

In [17]:
import plotly.subplots as sp
import plotly.graph_objects as go

In [18]:
from scipy import stats

In [19]:
import pandas as pd

# Import reads from google drive, three samples treated with energy metabolism inhibitors and one untreated

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
SRR26046013_DM_AOA_INH = sc.read_h5ad('/content/drive/MyDrive/Colab Notebooks/SRR26046013_Annotated_data_matrix.h5ad')

In [22]:
SRR26046013_DM_AOA_INH

AnnData object with n_obs × n_vars = 13546 × 0
    obs: 'n_fragment', 'frac_dup', 'frac_mito'
    uns: 'reference_sequences'
    obsm: 'fragment_paired'

In [23]:
SRR26046019_DM_UT = sc.read_h5ad('/content/drive/MyDrive/Colab Notebooks/SRR26046019_Annotated_data_matrix.h5ad')

In [24]:
SRR26046019_DM_UT

AnnData object with n_obs × n_vars = 10448 × 0
    obs: 'n_fragment', 'frac_dup', 'frac_mito'
    uns: 'reference_sequences'
    obsm: 'fragment_paired'

# Preprocess AD objects

In [25]:
gene_anno = '/content/drive/MyDrive/Colab Notebooks/gencode.v41.basic.annotation.gff3.gz'

# Create cell-by-gene matrix (replace with cell-by-bin if appropriate)
gene_matrix_1 = snap.pp.make_gene_matrix(SRR26046013_DM_AOA_INH, gene_anno=gene_anno)
gene_matrix_2 = snap.pp.make_gene_matrix(SRR26046019_DM_UT, gene_anno=gene_anno)

In [26]:
gene_matrix_1.X

<13546x60606 sparse matrix of type '<class 'numpy.uint32'>'
	with 73519275 stored elements in Compressed Sparse Row format>

In [27]:
#Check for suitable format
print(gene_matrix_1.X.dtype)  # Should output: int32, int64, or uint32

uint32


In [28]:
#Check that var contains gene names
print(gene_matrix_1.var_names)  # Should show gene names/IDs

Index(['DDX11L1', 'WASH7P', 'MIR6859-1', 'MIR1302-2HG', 'MIR1302-2', 'FAM138A',
       'OR4G4P', 'OR4G11P', 'OR4F5', 'ENSG00000238009',
       ...
       'MT-ND4', 'MT-TH', 'MT-TS2', 'MT-TL2', 'MT-ND5', 'MT-ND6', 'MT-TE',
       'MT-CYB', 'MT-TT', 'MT-TP'],
      dtype='object', length=60606)


# Add experiment index to cell barcode

In [29]:
gene_matrix_1.obs.index = gene_matrix_1.obs.index + '_1'

In [30]:
print(gene_matrix_1.obs.head())  # Shows a preview of cell-level info

                    n_fragment  frac_dup  frac_mito
AAAAAAAAAAAAAAAA_1         453  0.002119   0.038217
AAACAACGAACGAGCA_1       23759  0.415605   0.000252
AAACAACGAAGAGGCT_1       15377  0.403037   0.002336
AAACAACGAAGTCGGA_1       19907  0.421316   0.001004
AAACAACGACGCACTG_1         203  0.458115   0.019324


In [31]:
gene_matrix_2.obs.index = gene_matrix_2.obs.index + '_2'

In [32]:
print(gene_matrix_2.obs.head())

                    n_fragment  frac_dup  frac_mito
AAAAAAAAAAAAAAAA_2         361  0.000000   0.008242
AAACAACGATAAGTAG_2       26068  0.285284   0.001800
AAACAACGATAGGTTC_2       12427  0.335221   0.001527
AAACAACGATCTATCT_2       25665  0.346743   0.000973
AAACAACGATGCGTGC_2       17103  0.337616   0.000760


Combine AD matrices, add Treatment information

# Perform checks of AnnData matrix

In [33]:
import numpy as np
import pandas as pd
import scanpy as sc
from scipy.sparse import csr_matrix, issparse


def custom_highly_variable_genes(adata, n_top_genes=5000):
    """Calculates highly variable genes using a custom approach."""
    # Filter mitochondrial genes
    adata.var['mt'] = adata.var_names.str.startswith('MT-')
    adata_filtered = adata[:, ~adata.var['mt']]

    # Keep genes with at least 10 counts in at least 5 cells
    #sc.pp.filter_genes(adata_filtered, min_cells=5)
    #sc.pp.filter_genes(adata_filtered, min_counts=10)


    # Replace infinite values with 0 in the sparse matrix directly
    if issparse(adata_filtered.X):
        adata_filtered.X.data = np.nan_to_num(adata_filtered.X.data, posinf=0, neginf=0)
    else:
        adata_filtered.X = np.nan_to_num(adata_filtered.X, posinf=0, neginf=0)

    # Calculate mean and variance
    mean = np.mean(adata_filtered.X.toarray(), axis=0)
    var = np.var(adata_filtered.X.toarray(), axis=0)

    # Calculate dispersion
    # Clip values to avoid division by zero or very small numbers
    mean_clipped = np.clip(mean, 1e-8, None)
    dispersion = var / mean_clipped

    # Filter out genes with zero or NaN dispersion
    valid_dispersion_mask = np.isfinite(dispersion) & (dispersion > 0)
    dispersion = dispersion[valid_dispersion_mask]
    mean = mean[valid_dispersion_mask]
    gene_indices = np.where(valid_dispersion_mask)[0]

    # Sort genes by dispersion
    sorted_indices = np.argsort(dispersion)[::-1]  # Descending order
    top_gene_indices = gene_indices[sorted_indices[:n_top_genes]]

    # Update adata.var['highly_variable']
    adata_filtered.var['highly_variable'] = False
    adata_filtered.var.iloc[top_gene_indices, adata_filtered.var.columns.get_loc('highly_variable')] = True

    return adata_filtered

In [34]:
gene_matrix_1_filtered = custom_highly_variable_genes(gene_matrix_1)

# Subset your data
gene_matrix_1_subset = gene_matrix_1_filtered[:, gene_matrix_1_filtered.var['highly_variable']]
gene_matrix_1_subset.obs['Treatment'] = 'Treated w/AOA'

<ipython-input-33-dff88e0ee2f9>:44: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata_filtered.var['highly_variable'] = False
<ipython-input-34-0ad127af3ab0>:5: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  gene_matrix_1_subset.obs['Treatment'] = 'Treated w/AOA'


In [35]:
gene_matrix_2_filtered = custom_highly_variable_genes(gene_matrix_2)

# Subset your data
gene_matrix_2_subset = gene_matrix_2_filtered[:, gene_matrix_2_filtered.var['highly_variable']]
gene_matrix_2_subset.obs['Treatment'] = 'Untreated'

<ipython-input-33-dff88e0ee2f9>:44: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata_filtered.var['highly_variable'] = False
<ipython-input-35-e3d7095645e8>:5: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  gene_matrix_2_subset.obs['Treatment'] = 'Untreated'


In [36]:
#MOdify barcode names to reflect experiment number
new_barcode_names_1 = gene_matrix_1_subset.obs_names + '_1'
new_barcode_names_1

Index(['AAAAAAAAAAAAAAAA_1_1', 'AAACAACGAACGAGCA_1_1', 'AAACAACGAAGAGGCT_1_1',
       'AAACAACGAAGTCGGA_1_1', 'AAACAACGACGCACTG_1_1', 'AAACAACGACGGTACC_1_1',
       'AAACAACGAGAAGGCT_1_1', 'AAACAACGAGTACGTT_1_1', 'AAACAACGATCGCTAA_1_1',
       'AAACACGCTACTCCGT_1_1',
       ...
       'TTTGGGATGCGATTAC_1_1', 'TTTGGGATGCGGCACT_1_1', 'TTTGGGATGCGGTTCG_1_1',
       'TTTGGGATGGCGGTTC_1_1', 'TTTGGGATGGTCAATC_1_1', 'TTTGGGATGTCCAGCG_1_1',
       'TTTGGGATGTGTTCCC_1_1', 'TTTGGGATGTTAGCTG_1_1', 'TTTGGGATGTTGCGTA_1_1',
       'TTTGTCTGTGTAGATC_1_1'],
      dtype='object', length=13546)

In [37]:
#Update barcode names
gene_matrix_1_subset.obs_names = new_barcode_names_1

In [38]:
#Modify barcode names to reflect experiment number
new_barcode_names_2 = gene_matrix_2_subset.obs_names + '_2'
new_barcode_names_2

Index(['AAAAAAAAAAAAAAAA_2_2', 'AAACAACGATAAGTAG_2_2', 'AAACAACGATAGGTTC_2_2',
       'AAACAACGATCTATCT_2_2', 'AAACAACGATGCGTGC_2_2', 'AAACAACGATTGCGCG_2_2',
       'AAACAACGCTTGCGCG_2_2', 'AAACACGCGGCGCTGT_2_2', 'AAACACGCTCGCAACC_2_2',
       'AAACACGCTCGGAATG_2_2',
       ...
       'TTTGGGAACTGACCTA_2_2', 'TTTGGGACCCGACATT_2_2', 'TTTGGGACTGACCTAG_2_2',
       'TTTGGGATGAACGCAG_2_2', 'TTTGGGATGAACGTAA_2_2', 'TTTGGGATGAAGTCTC_2_2',
       'TTTGGGATGATGTGGG_2_2', 'TTTGGGATGCTCACGG_2_2', 'TTTGGGATGCTCTGGG_2_2',
       'TTTGGGTGTGTAGATC_2_2'],
      dtype='object', length=10448)

In [39]:
#Update barcode names
gene_matrix_2_subset.obs_names = new_barcode_names_2

In [40]:
adata_combined = gene_matrix_1_subset.concatenate(gene_matrix_2_subset, batch_key="Treatment", index_unique=None)

<ipython-input-40-96161036905d>:1: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  adata_combined = gene_matrix_1_subset.concatenate(gene_matrix_2_subset, batch_key="Treatment", index_unique=None)


In [41]:
count_df = pd.DataFrame(data=adata_combined.X.toarray(), index=adata_combined.obs_names, columns=adata_combined.var_names)
metadata_df = pd.DataFrame(data=adata_combined.obs['Treatment'], index=adata_combined.obs_names, columns=['Treatment'])

In [42]:
count_df

,ENSG00000230021,ENSG00000278791,MTATP6P1,MTCO3P12,AGRN,C1orf159,ACAP3,MIB2,PRKCZ,SKI,...,ARHGEF6,ENSG00000241489,EOLA2,EOLA2-DT,HCFC1,TMEM187,FLNA,EMD,G6PD,GAB3
AAAAAAAAAAAAAAAA_1_1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACAACGAACGAGCA_1_1,1,0,0,0,2,1,0,0,3,4,...,3,3,1,4,3,2,2,2,1,0
AAACAACGAAGAGGCT_1_1,0,0,0,0,2,0,1,0,1,6,...,5,1,0,1,0,0,1,0,1,2
AAACAACGAAGTCGGA_1_1,2,0,0,0,5,1,0,1,0,2,...,2,2,1,2,0,0,0,1,1,0
AAACAACGACGCACTG_1_1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGGGATGAAGTCTC_2_2,0,0,0,0,4,2,0,2,1,4,...,0,0,0,0,2,3,3,1,1,3
TTTGGGATGATGTGGG_2_2,0,0,0,0,2,0,0,1,3,6,...,0,2,1,2,4,4,2,1,2,1
TTTGGGATGCTCACGG_2_2,0,0,0,0,1,1,0,0,1,8,...,3,0,0,1,1,0,3,1,0,1
TTTGGGATGCTCTGGG_2_2,2,0,0,0,2,2,6,5,4,6,...,5,4,2,2,1,2,2,1,6,2


In [43]:
metadata_df

,Treatment
AAAAAAAAAAAAAAAA_1_1,0
AAACAACGAACGAGCA_1_1,0
AAACAACGAAGAGGCT_1_1,0
AAACAACGAAGTCGGA_1_1,0
AAACAACGACGCACTG_1_1,0
...,...
TTTGGGATGAAGTCTC_2_2,1
TTTGGGATGATGTGGG_2_2,1
TTTGGGATGCTCACGG_2_2,1
TTTGGGATGCTCTGGG_2_2,1


In [44]:
from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats

In [45]:
dds = DeseqDataSet(
    counts=count_df,
    metadata=metadata_df,
    design_factors="Treatment",
    refit_cooks=True,
)

In [46]:
!pip install tqdm

In [47]:
from tqdm.notebook import tqdm

In [48]:
import cProfile
import pstats

profiler = cProfile.Profile()
profiler.enable()

# Your DESeq2 analysis code here:
dds.deseq2()
results = dds.get_results()

profiler.disable()
stats = pstats.Stats(profiler).sort_stats('cumulative')
stats.print_stats()

Fitting size factors...
/usr/local/lib/python3.10/dist-packages/pydeseq2/dds.py:490: UserWarning: Every gene contains at least one zero, cannot compute log geometric means. Switching to iterative mode.
  self.fit_size_factors()
Fitting dispersions...


KeyboardInterrupt: 